## Imports

In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from scipy.stats.stats import pearsonr
import missingno as msno
from sklearn.cluster import KMeans
from matplotlib import gridspec
from sklearn.preprocessing import MinMaxScaler
import io
import requests
import os
from scipy.stats import mode
from scipy.spatial.distance import pdist
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from scipy.cluster.hierarchy import linkage, dendrogram
from collections import defaultdict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler

ModuleNotFoundError: No module named 'mlxtend'

In [5]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [2]:
# training_transformed contains the transformed (missing+transformation) dataset
dataset = 'training_transformed_new'

# donwload the dataset file
filename = dataset + '.pickle'
if not os.path.exists(filename):
  url = 'http://deselmo.com/dm/' + filename
  r = requests.get(url)
  with open(filename, 'wb') as f:
      f.write(r.content)
df_training_ = pd.read_pickle(filename)

In [3]:
# testing_transformed contains the transformed (transformation) dataset
dataset = 'test_transformed_new'

# donwload the dataset file
filename = dataset + '.pickle'
if not os.path.exists(filename):
  url = 'http://deselmo.com/dm/' + filename
  r = requests.get(url)
  with open(filename, 'wb') as f:
      f.write(r.content)
df_test_ = pd.read_pickle(filename)

In [4]:
df_training_.head()

,RefId,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,...,Variation_rate_RCP,Variation_rate_mean,VehBCost_Current,Difference_Current_AAP,Difference_Current_ACP,Difference_Current_RAP,Difference_Current_RCP,Markup_Current_AP_max,Markup_Current_CP_max,PurchYear
0,326,1,10/25/2010,ADESA,2008,2,KIA,SPECTRA,EX,4D SEDAN EX,...,2.263549,4.835677,29497.628673,528.0,1435.0,3505.0,4550.0,4033.0,5985.0,2010
1,42991,0,5/27/2009,MANHEIM,2006,3,DODGE,STRATUS V6 2.7L V6 M,SXT,4D SEDAN SXT FFV,...,2.150086,2.783101,11132.404224,1480.0,2461.0,2418.0,3478.0,3898.0,5939.0,2009
2,55273,0,3/18/2010,OTHER,2008,2,DODGE,CALIBER,SE,4D WAGON,...,2.733622,6.614564,49609.230335,-3.0,602.0,3221.0,3850.0,3218.0,4452.0,2010
3,29058,0,6/8/2009,OTHER,2005,4,FORD,FREESTAR FWD V6 3.9L,S,PASSENGER 3.9L,...,2.593055,16.058093,75874.487774,92.0,1933.0,977.0,2966.0,1069.0,4899.0,2009
4,34991,0,3/4/2009,MANHEIM,2005,4,CHRYSLER,TOWN & COUNTRY FWD V,Bas,MINIVAN 3.3L,...,3.939819,-0.571170,-3238.535646,-300.0,1141.0,630.0,2186.0,330.0,3327.0,2009


## SetUp

### Columns

In [ ]:
target = 'IsBadBuy'

categorical = [
  # 'PurchDate',
  # 'Auction',
  # 'Transmission',
  # 'WheelTypeID',
  # 'PRIMEUNIT',
  # 'BYRNO',
  # 'VNZIP1',
  # 'IsTopThreeAmericanName',

  'Make',
  'Model',
  'Trim',
  'SubModel',
  'Color',
  'WheelType',
  'Nationality',
  'Size',
  'TopThreeAmericanName',
  'AUCGUART',
  'VNST',
  'IsOnlineSale',
  'IsTransmissionAuto',
]

numerical = [
  'PurchYear',
  'VehYear',

  'VehicleAge',
  'VehOdo',
  'VehBCost',
  'WarrantyCost',

  'Markup_Current_AP_max',
  'Markup_Current_CP_max',
  'Markup_Acquisition_AP_max',
  'Markup_Acquisition_CP_max',

  'MMRAcquisitionAuctionAveragePrice',
  'MMRAcquisitionAuctionCleanPrice',
  'MMRAcquisitionRetailAveragePrice',
  'MMRAcquisitonRetailCleanPrice',
  'MMRCurrentAuctionAveragePrice',
  'MMRCurrentAuctionCleanPrice',
  'MMRCurrentRetailAveragePrice',
  'MMRCurrentRetailCleanPrice',

  'Difference_Acquisition_AAP',
  'Difference_Acquisition_ACP',
  'Difference_Acquisition_RAP',
  'Difference_Acquisition_RCP',
  'Difference_Current_AAP',
  'Difference_Current_ACP',
  'Difference_Current_RAP',
  'Difference_Current_RCP',
]

columns2use = [target] + categorical + numerical

In [ ]:
df_training = df_training_[columns2use].copy()
df_test = df_test_[columns2use].copy()

In [ ]:
label_encoders = dict()
column2encode = categorical

for col in column2encode:
    le = LabelEncoder()
    df_training[col] = le.fit_transform(df_training[col])
    label_encoders[col] = le

for col in column2encode:
    le = LabelEncoder()
    df_test[col] = le.fit_transform(df_test[col])
    label_encoders[col] = le

In [ ]:
df_test.head()

### Filter with dbscan

In [ ]:
filename = 'dbscan_labels.npy'
if not os.path.exists(filename):
  url = 'http://deselmo.com/dm/' + filename
  r = requests.get(url)
  with open(filename, 'wb') as f:
      f.write(r.content)
dbscan_labels = np.load(filename)

dbscan_noise_mask = np.vectorize(lambda x: x==0)(dbscan_labels)

df_training = df_training[dbscan_noise_mask].copy()
len(df_training)

### Create samples

In [ ]:
attributes = [col for col in columns2use if col != target]

X_train = df_training[attributes].values
y_train = df_training[target]

X_test = df_test[attributes].values
y_test = df_test[target]

## Training

In [ ]:
candidates = []

param_list = {
    'max_depth': [None] + list(np.arange(2, 50)),
    'min_samples_split': [2, 5, 10, 15, 20, 30, 50, 100, 150, 200, 250, 300],
    'min_samples_leaf': [1, 5, 10, 15, 20, 30, 50, 100, 150, 200],
    'criterion': ['gini', 'entropy'],
}

clf = DecisionTreeClassifier(criterion='gini', class_weight='balanced')

random_search = RandomizedSearchCV(clf, param_distributions=param_list, n_iter=100, verbose=1, cv=10, n_jobs=8)
random_search.fit(X_train, y_train)

report(random_search.cv_results_, n_top=3)

In [ ]:
# params = dict(min_samples_split=5, min_samples_leaf=1, max_depth=25, criterion='gini')
params = dict(min_samples_split=2, min_samples_leaf=1, max_depth=28, criterion='entropy')
# params = dict(min_samples_split=15, min_samples_leaf=1, max_depth=27, criterion='entropy')
# params = dict(min_samples_split=20, min_samples_leaf=1, max_depth=33, criterion='gini')

clf = DecisionTreeClassifier(class_weight='balanced', **params)
clf = clf.fit(X_train, y_train)

y_pred_tr = clf.predict(X_train)
print('Train Accuracy %s' % accuracy_score(y_train, y_pred_tr))
print('Train F1-score %s' % f1_score(y_train, y_pred_tr, average=None))
print('Train roc_auc_score %s' % roc_auc_score(y_train, y_pred_tr))

print()

y_pred_ts = clf.predict(X_test)
print('Test Accuracy %s' % accuracy_score(y_test, y_pred_ts))
print('Test F1-score %s' % f1_score(y_test, y_pred_ts, average=None))
print('Test roc_auc_score %s' % roc_auc_score(y_test, y_pred_ts))


In [ ]:
for col, imp in zip(columns2use[1:], clf.feature_importances_):
    print(col, imp)